# Custom Chatbot Project

__In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task__

In this project there were utilized 101 facts about Premier League from season 2022/23 (from `theanalyst.com` page). This dataset is appropiate, beacause we want to have a tool that will behave as an expert and will answer for questions about this league and that season.

There was utlized _Retrieval Augmented Generation_ technique to suplement prompt with context that should allow to answer for questions.

In [35]:
OPENAI_API_KEY = 'PUT HERE YOU OPENAPI KEY' 

SOURCE_URL = 'https://theanalyst.com/eu/2023/05/101-best-premier-league-facts-2022-23'
PAGE_FILEPATH = './facts.html'
CSV_FILEPATH_WITH_EMBEDDINGS = './facts_with_embeddings.csv'

EMBEDDING_MODEL = 'text-embedding-3-small'
COMPLETION_MODEL = 'gpt-3.5-turbo'

## Data Wrangling

In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
def pull_html_page(url: str):
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.content
    else:
        raise Exception('Connection error')


with open(PAGE_FILEPATH, mode='wb') as html_file:
    html_page = pull_html_page(SOURCE_URL)
    html_file.write(html_page)

## Read page

In [5]:
with open(PAGE_FILEPATH) as fp:
    soup = BeautifulSoup(fp, 'html.parser')

In [6]:
root_dom_node = soup.find('h2', {'class':'has-text-align-center wp-block-heading'})
root_dom_node

<h2 class="has-text-align-center wp-block-heading"><strong>August</strong></h2>

In [7]:
month_headers = [month_header.find_next('strong') for month_header in soup.find_all('h2', {'class':'has-text-align-center wp-block-heading'})]
month_headers

[<strong>August</strong>,
 <strong>September</strong>,
 <strong>October</strong>,
 <strong>November</strong>,
 <strong>December</strong>,
 <strong>January</strong>,
 <strong>February</strong>,
 <strong>March</strong>,
 <strong>April</strong>,
 <strong>May</strong>]

In [8]:
current_month = None
data = []

for node in root_dom_node.find_all_next():
    if node in month_headers:
        current_month = node.text
    elif node.name == 'ul':
        data.append(f"{current_month} 2024 -- {node.find_next('li').text.strip()}")

In [9]:
import pandas as pd

pd.set_option('display.max_colwidth', None)  
pd.set_option('display.max_rows', None)  

df = pd.DataFrame(data, columns=['text'])
df

,text
0,"August 2024 -- On 13 August 2022, Manchester City ended the day top and Manchester United ended the day bottom of the top-flight table for the first time since 29 November 1929."
1,August 2024 -- Erik ten Hag became the first manager to lose each of his first two games in charge of Manchester United since John Chapman in November 1921.
2,"August 2024 -- Harry Kane netted his 185th Premier League goal for Tottenham Hotspur against Wolves, overtaking Sergio Aguero’s record for Premier League goals for a single club (184 for Manchester City)."
3,August 2024 -- Brenden Aaronson’s opening goal in Leeds’ 3-0 win against Chelsea was the first time an American player scored under an American manager (Jesse Marsch) in Premier League history.
4,"August 2024 -- Darwin Núñez came off the bench to score and assist on his Premier League debut for Liverpool against Fulham, only the third player to score and assist as a substitute on debut, along with Sergio Aguero (2011) and Alvaro Morata (2017)."
5,"August 2024 -- Liverpool beat Bournemouth 9-0 in August to become the joint biggest win in Premier League history, equalling three other 9-0 wins: Man Utd 9-0 Ipswich (1995), Southampton 0-9 Leicester (2019) and Man Utd 9-0 Southampton (2021)."
6,"August 2024 -- Nottingham Forest’s starting XI against Tottenham was made up of players who were all British, the first entirely British and Northern Irish starting XI for a Premier League game since Blackpool’s against Man Utd in May 2011."
7,"August 2024 -- Jefferson Lerma scored after one minute and 58 seconds on the opening day for Bournemouth against Aston Villa, the quickest goal on MD1 by a newly promoted side in Premier League history."
8,"September 2024 -- Due to the postponements of Premier League fixtures in September following the death of Queen Elizabeth II and international fixtures taking place at the end of the month, only 18 Premier League games were played this month, the fewest games ever played in the month of September in a top-flight season."
9,"September 2024 -- In their 3-1 win against Arsenal, Manchester United’s Antony became the youngest Brazilian to score on his Premier League debut (22 years 192 days old). He also became the 100th Brazilian to play in the Premier League."


## Create Embedding Database

In [10]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [11]:
from typing import List, Union

BATCH_SIZE = 25

pd.reset_option('display.max_colwidth')  
pd.reset_option('display.max_rows')  

def get_embeddings(prompt: Union[str, List[str]], embedding_model: str) -> List[List[float]]:
    response = client.embeddings.create(
            input=prompt if type(prompt) is list else [prompt],
            model=embedding_model
    )
    return [row.embedding for row in response.data]
                                                                                     

def create_embeddings(df, embedding_model_name: str = EMBEDDING_MODEL, batch_size: int = 25) -> List[List[float]]:
    output = []
    for idx in range(0, len(df), BATCH_SIZE):
        batch = df.iloc[idx:idx+BATCH_SIZE].tolist()
        embeddings = get_embeddings(batch, embedding_model_name)
        output.extend(embeddings)

    return output

df['embedding'] = create_embeddings(df['text'])
df.to_csv(CSV_FILEPATH_WITH_EMBEDDINGS, sep=',', index=False)    
df

,text,embedding
0,"August 2024 -- On 13 August 2022, Manchester C...","[-0.01702962815761566, 0.007488281931728125, 0..."
1,August 2024 -- Erik ten Hag became the first m...,"[-0.05475905165076256, -0.011441872455179691, ..."
2,August 2024 -- Harry Kane netted his 185th Pre...,"[0.0049393996596336365, -0.016499018296599388,..."
3,August 2024 -- Brenden Aaronson’s opening goal...,"[0.005350820254534483, -0.006501710508018732, ..."
4,August 2024 -- Darwin Núñez came off the bench...,"[-0.055598579347133636, 0.012705056928098202, ..."
...,...,...
96,May 2024 -- Chelsea’s starting XI against Man ...,"[0.025021368637681007, -0.003792672883719206, ..."
97,May 2024 -- Erik ten Hag became just the fifth...,"[-0.07281055301427841, 0.010384906083345413, 0..."
98,May 2024 -- Harry Kane’s brace against Leeds o...,"[0.0240727998316288, 0.013070138171315193, 0.0..."
99,May 2024 -- Having also netted 30 goals in 201...,"[0.009395512752234936, 0.03216593712568283, 0...."


## Custom Query Completion

In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [12]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [13]:
from scipy.spatial.distance import cosine
from typing import List, Union


def build_simple_prompt(question: str):
    return [
        {
            'role': 'user',
            'content': question
        }
    ]

def build_custom_prompt(question: str, database_df):
    return [
        {
            'role': 'system',
            'content': """
            Anser the question based on provided context below. If the question cannot be answered based on provided context, say "I don't know the answer". We have 2024. Context contains facts from season 2022/2023 for English Premier League. Facts are annotated with date and seperated by lines. 
            Context: 
                {}
            """.format('\n\n'.join(build_custom_context(question, database_df)))
        },
        {
            'role': 'user',
            'content': question
        }
    ]

def build_custom_context(question: str, database_df: df, n: int = 5):
    question_embedding = get_embeddings(question, EMBEDDING_MODEL)[0]
    
    df = database_df.copy()
    df["distances"] = df['embedding'].apply(lambda embedding: cosine(embedding, question_embedding))

    df.sort_values("distances", ascending=True, inplace=True)
    return df.iloc[:n]['text'].tolist()


def get_embeddings(prompt: Union[str, List[str]], embedding_model: str) -> List[List[float]]:
    response = client.embeddings.create(
            input=prompt if type(prompt) is list else [prompt],
            model=embedding_model
    )
    return [row.embedding for row in response.data]



In [18]:
def handle_question(prompt, model_name: str = COMPLETION_MODEL):
    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        max_tokens=100
    )
    return response.choices[0].message.content
    

## Custom Performance Demonstration

In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

In [19]:
import pandas as pd

df = pd.read_csv(CSV_FILEPATH_WITH_EMBEDDINGS)
df['embedding'] = df['embedding'].apply(lambda value: [float(dim) for dim in value.replace('[', '').replace(']', '').split(',')])

### Question 1

In [25]:
question = 'Who did win the Premier League in season 2022/2023?'
print('__Answer:__', handle_question(build_simple_prompt(question)))
print('__Answer with Context:__', handle_question(build_custom_prompt(question, df)))

__Answer:__ As an AI language model, I don't have real-time data or the ability to browse the internet. Therefore, I cannot provide you with information about future events. Please refer to credible sports news sources or visit the official Premier League website for the most up-to-date information on the winner of the Premier League in the 2022/2023 season.
__Answer with Context:__ I don't know the answer.


### Question 2

In [26]:
question = 'What football team did Harry Kane play in in season 2022/2023?'
print('__Answer:__', handle_question(build_simple_prompt(question)))
print('__Answer with Context:__', handle_question(build_custom_prompt(question, df)))

__Answer:__ Sorry, but I can't provide the answer as the 2022/2023 season is in the future and hasn't happened yet.
__Answer with Context:__ Harry Kane played for Tottenham Hotspur in the 2022/2023 season.


### Question 3

In [30]:
question = 'What team did finish the match with the most competitive win? What was the result? Who was the opponent?'
print('__Answer:__', handle_question(build_simple_prompt(question)))
print('__Answer with Context:__', handle_question(build_custom_prompt(question, df)))

__Answer:__ I'm sorry, but as an AI language model, I don't have access to real-time data or the ability to browse the internet. Therefore, I cannot provide you with the specific information about the most recent competitive match and its outcome. I recommend checking sports news or websites that cover the particular sport you are interested in to find the most recent match results.
__Answer with Context:__ Liverpool finished the match with the most competitive win. The result of the match was 7-0. The opponent was Manchester United.


## More questions

In [34]:
counter = 1
while True:
    question = input(f'#{counter} What do you want to find out about Premier League from season 2022/2023?')
    print('__Answer:__', handle_question(build_simple_prompt(question)))
    print('__Answer with Context:__', handle_question(build_custom_prompt(question, df)))
    counter += counter

#1 What do you want to find out about Premier League from season 2022/2023? Who was the coach of Manchester City in 2022/2023?


__Answer:__ As an AI language model, I don't have real-time data, but based on the information you provided, I cannot accurately determine the coach of Manchester City in the specific season of 2022/2023. It is best to refer to reliable sources or conduct an online search for the latest information.
__Answer with Context:__ The coach of Manchester City in the 2022/2023 season was Pep Guardiola.


KeyboardInterrupt: Interrupted by user